<a href="https://colab.research.google.com/github/temimujidat/Vibeoo/blob/main/Podcast_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors

import numpy as np

from bson.objectid import ObjectId
from datetime import datetime

import pymongo as pm
import requests
import pprint
from pymongo import MongoClient

In [6]:
from google.colab import files

upload = files.upload()

Saving playerlists.csv to playerlists.csv


In [19]:
users_pd = pd.read_csv('users.csv')
shows_pd = pd.read_csv('shows.csv')
episode_pd = pd.read_csv('episode.csv')
playlist = pd.read_csv('playerlists.csv')

In [20]:
playlist.head()

,playlist_id,episode_id,user_id,createdAt
0,1,3,9,42:30.2
1,2,35,9,43:08.2
2,3,31,9,48:46.7
3,4,32,9,48:59.3
4,5,38,1,49:19.4


In [21]:
playlist.shape

(24, 4)

In [22]:
playlist.describe()

,playlist_id,episode_id,user_id
count,24.000000,24.000000,24.000000
mean,12.500000,20.666667,7.250000
std,7.071068,14.008279,2.574963
min,1.000000,2.000000,1.000000
25%,6.750000,5.000000,5.500000
50%,12.500000,20.500000,8.000000
75%,18.250000,34.250000,9.000000
max,24.000000,39.000000,11.000000


In [23]:
#playlist[playlist["user_id"]==9]

In [24]:
playlist["createdAt"] = 1

In [25]:
playlist = playlist.rename({"createdAt":"rating"}, axis = "columns")

In [26]:
merged = playlist.merge(episode_pd, left_on="episode_id", right_on="episodeId")

In [27]:
merged = merged[["user_id", "episodeTitle", "rating"]]

In [28]:
merged.head()

,user_id,episodeTitle,rating
0,9,Mirror on the Car,1
1,7,Mirror on the Car,1
2,9,davido or wizkid for president,1
3,4,davido or wizkid for president,1
4,9,the power of vision,1


In [29]:
piv = merged.pivot_table(index="user_id", columns="episodeTitle", values="rating", aggfunc=sum)
#tpiv = piv
piv

episodeTitle,Football Night,Mirror on the Car,Mirror on the Door,Powerhouses Meet,Ucl Final,davido or wizkid for president,dealing with discouragement,dealing with doubt,fruit bashing,making your life better through descipline,making your life better through desire,making your life better through diligence,making your life better through divine influence,the power of vision,top 10 trendy by october,top ten record label in naija,understanding mental illness,we call it podding,why not exercise more often
user_id,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,NaN
6,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1.0,1.0,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0


In [30]:
piv.fillna(0, inplace=True)
piv = piv.T
piv.head()

user_id,1,4,6,7,9,11
episodeTitle,,,,,,
Football Night,0.0,0.0,0.0,1.0,1.0,0.0
Mirror on the Car,0.0,0.0,0.0,1.0,1.0,0.0
Mirror on the Door,0.0,0.0,1.0,0.0,0.0,0.0
Powerhouses Meet,0.0,1.0,0.0,0.0,0.0,0.0
Ucl Final,0.0,0.0,0.0,1.0,1.0,0.0


In [31]:
piv = piv.loc[:, (piv != 0).any(axis=0)]
piv.shape

(19, 6)

In [32]:
import scipy as sp
from scipy.sparse import csr_matrix
import operator
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import random
#import implicit

In [33]:
piv_sparse = sp.sparse.csr_matrix(piv.values)
piv.reset_index(inplace=True)

In [34]:
episode_similarity = (piv_sparse)
user_similarity = cosine_similarity(piv_sparse.T)

In [41]:
episode_similarity

<19x6 sparse matrix of type '<class 'numpy.float64'>'
	with 24 stored elements in Compressed Sparse Row format>

In [40]:
episode_sim_df = pd.DataFrame(episode_similarity, index=piv.index, columns=piv.index)
#user_sim_df = pd.DataFrame(user_similarity, index=piv.columns, columns=piv.columns)

ValueError: ignored